2.2 Mỗi file .md cần có tiêu đề h1 `#`
next: tạo filelist 1make_filelist.ipynb

In [ ]:
import re
from pathlib import Path
import sys

def modify_header_line(folder_path_str):
    """
    Quét tất cả các file .md trong thư mục được chỉ định và thay đổi
    dòng đầu tiên từ định dạng '**(string)**' thành '# (string)'.

    Args:
        folder_path_str (str): Đường dẫn đến thư mục chứa các file .md.
    """
    try:
        folder_path = Path(folder_path_str)
        # Kiểm tra xem đường dẫn có tồn tại và có phải là thư mục không
        if not folder_path.is_dir():
            print(f"Lỗi: Đường dẫn '{folder_path_str}' không tồn tại hoặc không phải là thư mục.")
            return
    except Exception as e:
        print(f"Lỗi khi truy cập đường dẫn '{folder_path_str}': {e}")
        return

    # Biểu thức chính quy để tìm dòng bắt đầu và kết thúc bằng '**'
    # ^     : Bắt đầu dòng
    # \*\*  : Ký tự '*' kép (cần escape bằng \ vì * là ký tự đặc biệt trong regex)
    # (.*?) : Nhóm chụp (capturing group 1) - khớp với bất kỳ ký tự nào (.)
    #         0 hoặc nhiều lần (*), một cách không tham lam (?)
    #         -> lấy nội dung ngắn nhất có thể giữa '**' và '**'
    # \*\*  : Ký tự '*' kép ở cuối
    # $     : Kết thúc dòng
    # --- Cập nhật: Sử dụng (.*) thường tốt hơn trong trường hợp này ---
    # (.*)  : Nhóm chụp (capturing group 1) - khớp với bất kỳ ký tự nào (.)
    #         0 hoặc nhiều lần (*) -> lấy tất cả nội dung giữa '**' và '**'
    pattern = re.compile(r"^\*\*(.*)\*\*$")
    pattern = re.compile(r"^##\s(.*)$")
    print(f"Đang quét thư mục: {folder_path.resolve()}") # In đường dẫn tuyệt đối

    modified_count = 0
    processed_count = 0

    # Lặp qua tất cả các mục trong thư mục
    for item in folder_path.iterdir():
        # Chỉ xử lý các file có đuôi .md (không phân biệt hoa thường)
        if item.is_file() and item.suffix.lower() == '.md':
            processed_count += 1
            print(f"\n--- Đang xử lý file: {item.name} ---")
            try:
                # Đọc tất cả các dòng của file, sử dụng encoding utf-8
                with open(item, 'r', encoding='utf-8') as f:
                    lines = f.readlines()

                # Nếu file trống, bỏ qua
                if not lines:
                    print("  -> Bỏ qua: File trống.")
                    continue

                # Lấy dòng đầu tiên và loại bỏ khoảng trắng thừa ở hai đầu để kiểm tra
                first_line_stripped = lines[0].strip()

                # Kiểm tra xem dòng đầu tiên có khớp với mẫu không
                match = pattern.match(first_line_stripped)

                if match:
                    # Lấy nội dung bên trong dấu ** (nhóm chụp 1)
                    inner_string = match.group(1)

                    # Tạo dòng mới với định dạng '# nội_dung'
                    # Giữ lại ký tự xuống dòng gốc nếu có
                    original_newline = '\n' if lines[0].endswith('\n') else ''
                    new_first_line = f"# {inner_string}{original_newline}"

                    print(f"  Dòng gốc: '{lines[0].strip()}'")
                    print(f"  Đổi thành: '{new_first_line.strip()}'")

                    # Ghi lại nội dung đã sửa đổi vào file (ghi đè)
                    with open(item, 'w', encoding='utf-8') as f:
                        f.write(new_first_line) # Ghi dòng đầu đã sửa
                        f.writelines(lines[1:]) # Ghi phần còn lại của file

                    modified_count += 1
                    print("  -> Đã sửa đổi file.")
                else:
                    print(f"  Dòng đầu tiên: '{lines[0].strip()}'")
                    print("  -> Bỏ qua (không khớp mẫu **string**).")

            except FileNotFoundError:
                print(f"Lỗi: Không tìm thấy file {item.name} (có thể đã bị xóa trong quá trình chạy?).")
            except IOError as e:
                print(f"Lỗi I/O khi xử lý file {item.name}: {e}")
            except Exception as e:
                # In lỗi chi tiết hơn để dễ debug
                print(f"Lỗi không xác định khi xử lý file {item.name}:")
                import traceback
                traceback.print_exc()


    print("\n--- Hoàn thành ---")
    print(f"Đã xử lý {processed_count} file .md.")
    print(f"Đã sửa đổi {modified_count} file.")


# Lấy đường dẫn thư mục từ người dùng
folder_to_scan = "../docs/kinhtangchi/sujato-en/"
modify_header_line(folder_to_scan)